In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 7, 13)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-07-13
yesterday: 2023-07-12


### Tables in the process

In [3]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [4]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,KTC,84,2023-06-15,900,50.00,1042,1263,2023-07-13,45.00,83.84,5.87,"40,410.29",28,"-4,689.38",-10.40%,-135.54%,-42,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [5]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)
name

'KTC'

In [6]:
sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1263



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,KTC,1263,84,2023-06-15,900,50.00,93.15,6.52,Sold,"45,099.67",DTD,None


In [7]:
transaction, name, qty, sell_price, buy_price

('S', 'KTC', -900, 45.0, 50.0)

### In case of buy, retreive buy id from latest buy record (id)

In [5]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,PTG,1273,163,2023-07-12,"3,600",11.40,84.95,5.95,Active,"41,130.90",DTD,None


In [6]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'PTG', 3600, 11.4)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [8]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'KTC'



,name,date,shares,price,active,period,grade
0,KTC,2023-06-15,900,50.00,1,4,A2


In [9]:
#qty = -30000
new_cost = 50
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + -900, price = 50
WHERE name = 'KTC'



In [10]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [11]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'KTC'



,name,date,shares,price,active,period,grade
0,KTC,2023-06-15,0,50.00,1,4,A2


### If shares become zero, must delete from buy table

In [12]:
sqlDel = """
DELETE FROM buy
WHERE name = '%s' AND volbuy = 0"""
sqlDel = sqlDel % name
print(sqlDel)


DELETE FROM buy
WHERE name = 'KTC' AND volbuy = 0


In [13]:
rp = const.execute(sqlDel)
rp.rowcount

1

### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [14]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'KTC'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual


In [13]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + -1800
WHERE name = 'JMART'



In [16]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [17]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'JMART'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,JMART,0.6600,0.0000,0.4000,0.0000,1.0600,"7,200",2023-04-11,2023-05-03,,0


### If shares become zero and want to delete from DIVIDEND table

In [15]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s'
"""
sqlDel = sqlDel % (name)
print(sqlDel)


DELETE FROM dividend
WHERE name = 'KTC'



In [16]:
rp = const.execute(sqlDel)
rp.rowcount

0

### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [17]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'KTC'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9796,KTC,0.00,0.00,I,48.00,52.50,0.00,0.00,50.00,900,-4,4,900,0,0,5pct,SET50


In [18]:
buy_target  = 40.50 # must calculate manually beforehand
sell_target = 50
cost = 0
status = 'O'
reason = "52WL"
sell_qty = 900

sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s,
qty = %s, buy_target = %s, sell_target = %s, cost = %s, reason = '%s', status = '%s'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, sell_qty, buy_target, sell_target, cost, reason, status, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + -900,
qty = 900, buy_target = 40.5, sell_target = 50, cost = 0, reason = '52WL', status = 'O'
WHERE name = 'KTC'



In [19]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [20]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'KTC'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9796,KTC,0.00,0.00,O,40.50,50.00,0.00,0.00,0.00,900,-4,4,0,0,0,52WL,SET50


### End of Update stocks table in SQLite port_lite database from sale transaction

### If shares become zero and no longer wants to keep STOCK record

In [24]:
sqlDel = """
DELETE FROM stocks
WHERE name = '%s' AND available_qty <= 0
"""
sqlDel = sqlDel % name
print(sqlDel)


DELETE FROM stocks
WHERE name = 'BANPU' AND available_qty <= 0



In [25]:
rp = conlite.execute(sqlDel)
rp.rowcount

1

## End of Sale process

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [7]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy['date'] = pd.to_datetime(buy['date'])
buy.drop(['volsell', 'volbal'], axis=1, inplace = True)
buy.rename(columns={'volbuy':'shares'}, inplace = True)
buy['shares'] = buy['shares'].astype('int64')
buy


SELECT * 
FROM buy 
WHERE name = 'PTG'



,name,date,shares,price,active,dividend,period,grade


### In case there is the same stock name in port

In [8]:
#qty = 1200
new_cost = 27.00 # new average cost must be calculated manually beforehand
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + 3600, price = 27.0
WHERE name = 'JMART'



In [9]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [10]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'JMART'



,name,date,shares,price,active,dividend,period,grade
0,JMART,2023-01-20,"10,800",27.00,1,1.0600,4,C3


### In case of totally new stock

In [8]:
rcd = ['PTG', '2023-07-12', 3600, 11.40, 0, 0, 1, 0, 4,'A2']
rcd

['PTG', '2023-07-12', 3600, 11.4, 0, 0, 1, 0, 4, 'A2']

In [9]:
sqlIns = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [10]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

In [11]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'PTG'



,name,date,shares,price,active,dividend,period,grade
0,PTG,2023-07-12,"3,600",11.40,1,0.0000,4,A2


### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [12]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'TFFIF' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,TFFIF,0.1032,0.1003,0.0920,0.1032,0.3987,0,2023-05-31,2023-06-19,,1


In [13]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 10000
WHERE name = 'TFFIF'



In [14]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [15]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'TFFIF' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,TFFIF,0.1032,0.1003,0.0920,0.1032,0.3987,"10,000",2023-05-31,2023-06-19,,1


### In case of totally new stock

In [ ]:
rcd = ['ASK', 1.44, 0, 0, 0, 1.44, 4500, '2023-02-28', '2023-04-27', '', 1]
rcd

In [ ]:
sqlIns = """
INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)

In [ ]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

### End of Create dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [12]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks


SELECT * 
FROM stocks 
WHERE name = 'PTG'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9800,PTG,0,0,O,11.4,0,0,0,0,1800,-4,4,0,0,0,,SET100


In [13]:
new_cost = 11.4
buy_target = 11.4
sell_target = 12.10
status = 'I'
reason = 'DTD'
new_buy_qty = 3600
new_sell_qty = 3600
new_cost, buy_target, sell_target, status, reason, new_buy_qty, new_sell_qty 

(11.4, 11.4, 12.1, 'I', 'DTD', 3600, 3600)

In [14]:
#qty = 12000
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, 
cost = %s, buy_target = %s, sell_target = %s, status = '%s', 
reason = '%s', qty = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, buy_target, sell_target, status, reason, new_sell_qty, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 3600, 
cost = 11.4, buy_target = 11.4, sell_target = 12.1, status = 'I', 
reason = 'DTD', qty = 3600
WHERE name = 'PTG'



In [15]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [16]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'PTG'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9800,PTG,0.00,0.00,I,11.40,12.10,0.00,0.00,11.40,"3,600",-4,4,"3,600",0,0,DTD,SET100


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process

In [21]:
sql = "SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '%s' \
ORDER BY period, buy.name"
sql = sql % (yesterday)
print(sql)

SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '2023-07-12' ORDER BY period, buy.name


In [22]:
output = pd.read_sql(sql, const)
output

,period,grade,name,date,volbuy,buy_price,mkt_price,amtbuy,amtmkt,amtpol,pctpol
0,1,C1,JASIF,2018-05-17,"130,000",10.00,6.95,"1,300,000.00","903,500.00","-396,500.00",-30.50
1,1,C4,MCS,2016-09-20,"78,000",15.10,6.25,"1,177,800.00","487,500.00","-690,300.00",-58.61
2,1,C1,PTTGC,2021-03-17,"6,000",64.75,39.00,"388,500.00","234,000.00","-154,500.00",-39.77
3,1,C2,TMT,2021-08-16,"36,000",10.20,7.25,"367,200.00","261,000.00","-106,200.00",-28.92
4,2,A4,ASK,2023-02-16,"4,500",31.00,24.40,"139,500.00","109,800.00","-29,700.00",-21.29
5,2,C4,ASP,2022-03-07,"30,000",3.80,2.76,"114,000.00","82,800.00","-31,200.00",-27.37
6,2,A2,CPNREIT,2022-08-16,"60,000",17.55,10.90,"1,053,000.00","654,000.00","-399,000.00",-37.89
7,2,C1,DIF,2020-08-01,"30,000",14.70,10.80,"441,000.00","324,000.00","-117,000.00",-26.53
8,2,B1,GVREIT,2022-08-24,"30,000",8.90,8.30,"267,000.00","249,000.00","-18,000.00",-6.74
9,2,B3,NER,2021-09-01,"27,000",7.45,4.44,"201,150.00","119,880.00","-81,270.00",-40.40


In [23]:
file_name = 'portfolio.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

output.to_csv(output_file, header=True, index=False)
output.to_csv(data_file, header=True, index=False)
output.to_csv(one_file, header=True, index=False)
output.to_csv(osd_file, header=True, index=False)